<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/master/Demos/Ch4_MultidimensionalIntegrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integration in multiple dimensions
Before, we talked about
$$\int_a^b f(x)\,dx$$
and now let's talk about
$$\int_a^b \int_c^d f(x,y)\,dy dx$$
and even triple or higher-dimensional integrals.

Our trick is to reduce this to 1D integrals:
$$\int_a^b \underbrace{\int_c^d f(x,y)\,dy}_{g(x)} dx = \int_a^b g(x)\,dx$$
where
$$g(x) = \int_c^d f(x,y)\,dy.$$

In [1]:
import numpy as np
from scipy.integrate import quadrature, quad, dblquad  # quadrature is Gassian quadrature
import functools # used for decorators

### Let's pick an integral
From exercise set 4.8 problem 1a in Burden and Faires, find
$$\int_{2.1}^{2.5} \int_{1.2}^{1.4} xy^2 \, dydx$$
which we can do in closed form
$$
\ldots = \int_{2.1}^{2.5} \frac13xy^3\big|_{1.2}^{1.4} dx
= \int_{2.1}^{2.5} \frac13x(1.4^3-1.2^3) dx
= \frac13\frac12(1.4^3-1.2^3)(2.5^2-2.1^2)
$$

Note that this is really a **separable** integral, meaning we can pull the $x$ out of the inner integrand, and then multiply two 1D integrals: $$f(x,y) = f(x)\cdot f(y)$$.  You generally cannot do this.

In [2]:
# lambda functions are more complicated to vectorize
# f = lambda x,y : x*y**2

#@np.vectorize  # not needed
def f(x,y):
  return x*y**2

a, b = 2.1, 2.5
c, d = 1.2, 1.4
I = (d**3-c**3)*(b**2-a**2)/6  # true value

Let's check that scipy's builtin 2D integrator, `dblquad`, works as intended

In [4]:
# Let's try getting it via scipy.  Be careful!  The input f must be:
# "A Python function or method of at least two variables: y must be the first argument and x the second argument."
def swap_decorator(f):
  @functools.wraps(f)
  def decorated(x,y):
    return f(y,x)
  return decorated

Q,errEst = dblquad( swap_decorator(f), a,b,c,d)
print(f'True integral {I:.10f}, estimate {Q:.10f}, error {abs(I-Q):.3e}')

True integral 0.3115733333, estimate 0.3115733333, error 1.665e-16


### Our procedure is going to rely on a 1D integrator
Call this the "base" quadrature rule. We can use whichever one we like. We may want to adjust the tolerance settings so that it runs faster (but is less accurate), as this will be important for very high dimensional integrals

In [9]:
def baseQuadratureRule(f,a,b):
  """1D integration of f(x) from a to b"""
  # Either do this Gaussian quadrature rule...
  # Q, errEstimate = quadrature(f,a,b,tol=1e-14,rtol=1e-14,maxiter=200)

  # or do this adaptive quadrature rule...
  Q, errEstimate = quad(f,a,b,epsabs=1.5e-8,epsrel=1.5e-8,limit=50)
  return Q

Make sure the base quadrature rule works

In [6]:
g = np.cos
G = np.sin
aa,bb = -1,2
Q = baseQuadratureRule( g,aa,bb )
print(f'Error of based quadrature rule is {abs(Q-G(bb)+G(aa)):.2e}')

Error of based quadrature rule is 2.22e-16


## Define the 2D quadrature rule

In [7]:
def my2D_quadrature(f,a,b,c,d, baseQuadRule ):
  """takes f(x,y) and integrates x from [a,b] and y from [c,d]"""
  
  @np.vectorize # not needed if using "quad", but needed for "quadrature"
  def g(x):
    """ g(x) = \int_c^d f(x,y) dy """
    #@np.vectorize # not needed
    def f_as_fcn_of_y(y):
      return f(x,y)

    # the limits are c to d, but you can see that if we wanted
    # them to be functions of x, it would be as simple
    # as calling  baseQuadRule( f_as_fcn_of_y, c(x), d(x) )
    Q = baseQuadRule( f_as_fcn_of_y, c, d )
    return Q
  
  Q = baseQuadRule( g, a, b )
  return Q

In [10]:
Q = my2D_quadrature(f,a,b,c,d,baseQuadratureRule)

print(f'True integral {I:.10f}, estimate {Q:.10f}, error {abs(I-Q):.3e}')

True integral 0.3115733333, estimate 0.3115733333, error 1.665e-16


Hopefully you see that it is conceptually easy, though it may require advanced **programming** tricks to get it to work.

Some of the base integrators will try to evaluate $f$ at a whole array of points all at once, expecting it to output an array (since this is usually much faster than a `for` loop).  If we have 2 inputs, this can cause a problem.  That's what some of the `np.vectorize` takes care of